In [ ]:
pip install --upgrade pip

In [ ]:
pip install -r requirements.txt

In [ ]:
pip install --upgrade  -q langchain langchain-community

In [ ]:
pip show openai

In [ ]:
pip show langchain

### Python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
# loading the API Keys from .env
load_dotenv(find_dotenv(), override=True)

os.environ.get('OPENAI_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()  

# invoking the llm (running the prompt)
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo', temperature=0.1)
print(output.content)

In [ ]:
from langchain.schema import (SystemMessage, AIMessage, HumanMessage)

messages = [SystemMessage(content="Your are a physicist and respond only in German."),
            HumanMessage(content="Explain quantum mechanics in one setence.")]
output = llm.invoke(messages)
print(output.content)

## Caching LLM Responses
### In-Memory Cache

In [11]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model='gpt-3.5-turbo-instruct')

In [ ]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a dad joke.'
llm.invoke(prompt)

In [ ]:
%%time
llm.invoke(prompt)

### SQL Caching

In [ ]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [ ]:
%%time
# First request (not in cache, takes longer)
llm.invoke("Tell me a joke")

In [ ]:
%%time
# Second request (cached, faster)
llm.invoke("Tell me a joke")

## LLM Streaming

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

In [ ]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

## PromptTemplates

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define a template for the prompt
template = '''You are an experienced virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template=template)

# Fill in the variable: virus and language
prompt = prompt_template.format(virus='hiv', language='german')
prompt  # Returns the generated prompt

In [ ]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

## ChatPromptTemplates


In [24]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

# Fill in the specific values for n and area
messages = chat_template.format_messages(n='5', area='World')
print(messages)  # Outputs the formatted chat messages

[SystemMessage(content='You respond only in the JSON format.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Top 5 countries in World by population.', additional_kwargs={}, response_metadata={})]


In [25]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

{
    "countries": [
        {
            "rank": 1,
            "name": "China",
            "population": "1,439,323,776"
        },
        {
            "rank": 2,
            "name": "India",
            "population": "1,380,004,385"
        },
        {
            "rank": 3,
            "name": "United States",
            "population": "331,002,651"
        },
        {
            "rank": 4,
            "name": "Indonesia",
            "population": "273,523,615"
        },
        {
            "rank": 5,
            "name": "Pakistan",
            "population": "220,892,340"
        }
    ]
}


## Simple Chains

In [26]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experience virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)
output = chain.invoke({'virus': 'HSV', 'language': 'Spanish'})
print(output)

/var/folders/33/tp9jds_x1fs4nbs5l28k2s3r0000gn/T/ipykernel_62833/1243800677.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(




> Entering new LLMChain chain...
Prompt after formatting:
You are an experience virologist.
Write a few sentences about the following virus "HSV" in Spanish.

> Finished chain.
{'virus': 'HSV', 'language': 'Spanish', 'text': 'El virus del herpes simple (HSV por sus siglas en inglés) es un virus de la familia Herpesviridae que afecta a los humanos. Hay dos tipos de HSV: el tipo 1, que generalmente causa herpes labial, y el tipo 2, que suele causar herpes genital. Este virus es altamente contagioso y se transmite a través del contacto directo con las lesiones activas o con las secreciones de una persona infectada. Una vez que una persona está infectada, el virus permanece latente en el cuerpo y puede reactivarse en momentos de estrés, fiebre u otros factores desencadenantes.'}


In [27]:

from langchain_core.output_parsers import StrOutputParser

chain = prompt_template | llm | StrOutputParser()
output = chain.invoke({'virus': 'HSV', 'language': 'Spanish'})
print(output)

El virus del herpes simple (HSV por sus siglas en inglés) es un virus de la familia Herpesviridae que afecta a los humanos. Hay dos tipos de HSV: el tipo 1, que generalmente causa herpes labial, y el tipo 2, que suele causar herpes genital. Este virus es altamente contagioso y se transmite a través del contacto directo con las lesiones activas o con las secreciones de una persona infectada. Una vez que una persona está infectada, el virus permanece latente en el cuerpo y puede reactivarse en momentos de estrés, fiebre u otros factores desencadenantes.


In [28]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points'
prompt_template = PromptTemplate.from_template(template=template)

# Initialize an LLMChain with the ChatOpenAI model and the prompt template
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

country = input('Enter Country: ')

# Invoke the chain with specific virus and language values
output = chain.invoke(country)
print(output['text'])



> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of France?. List the top 3 places to visit in that city. Use bullet points

> Finished chain.
The capital of France is Paris.

Top 3 places to visit in Paris:
- Eiffel Tower
- Louvre Museum
- Notre-Dame Cathedral


## Sequential Chains


In [29]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Initialize the first ChatOpenAI model (gpt-3.5-turbo) with specific temperature
llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

# Define the first prompt template
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)
# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

# Initialize the second ChatOpenAI model (gpt-4-turbo) with specific temperature
llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)

# Define the second prompt template
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
# Create another LLMChain using the second model and the prompt template
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('linear regression')




> Entering new SimpleSequentialChain chain...
Sure! Here is a simple implementation of linear regression in Python:

```python
import numpy as np

def linear_regression(x, y):
    n = len(x)
    sum_x = np.sum(x)
    sum_y = np.sum(y)
    sum_xy = np.sum(x*y)
    sum_x_squared = np.sum(x**2)
    
    m = (n*sum_xy - sum_x*sum_y) / (n*sum_x_squared - sum_x**2)
    b = (sum_y - m*sum_x) / n
    
    return m, b

# Example usage
x = np.array([1, 2, 3, 4, 5])
y = np.array([2, 4, 5, 4, 5])

m, b = linear_regression(x, y)
print(f"The equation of the line is y = {m}x + {b}")
```

This function calculates the slope (m) and y-intercept (b) of the line that best fits the given data points using the method of least squares. You can use this function to perform linear regression on any set of data points.
The provided Python function `linear_regression(x, y)` calculates the coefficients (slope and y-intercept) for a line that best fits a set of data points (`x`, `y`) according to the method of l

In [30]:
pip install -q --upgrade langchain_experimental

2742.84s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(1, 100) if n % 13 == 0])')

Python REPL can execute arbitrary code. Use with caution.


'[13, 26, 39, 52, 65, 78, 91]\n'

In [2]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model with gpt-4-turbo and a temperature of 0
llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

# Create a Python agent using the ChatOpenAI model and a PythonREPLTool
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

# Invoke the agent
prompt = 'Calculate the square root of the factorial of 12 and display it with 4 decimal points'
agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
To solve this, I need to calculate the factorial of 12 first, then find its square root, and finally format the result to display it with 4 decimal points. I will use Python's math module for both the factorial and square root calculations, and then format the output as required.

Action: Python_REPL
Action Input: import math
print(f"{math.sqrt(math.factorial(12)):.4f}")
Observation: 21886.1052

Thought:I now know the final answer
Final Answer: 21886.1052

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [3]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...
I need to calculate 5.1 raised to the power of 7.3 to get the answer.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [4]:
response

{'input': 'What is the answer to 5.1 ** 7.3?', 'output': '146306.05007233328'}